In [32]:
import torch
import torch.nn.functional as F
import numpy.random as nprandom
import  random

## This comes from cs230 Stanford

## common structure for PyTorch work

data/
experiments/
model/
    net.py
    data_loader.py
train.py
evaluate.py
search_hyperparams.py
synthesize_results.py
evaluate.py
utils.py

In [33]:
%timeit a = random.expovariate(1.0)
%timeit b = nprandom.exponential(1.0)

304 ns ± 0.724 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.19 µs ± 2.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
a = [3,4]
b = [2,6]
a = b if a<b else a.pop()
a

4

In [69]:
x = torch.ones(2,2,requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [70]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [72]:
y.grad_fn

In [73]:
z = y*y*3
out = z.mean()
z, out

(tensor([[27., 27.],
         [27., 27.]], grad_fn=<MulBackward0>),
 tensor(27., grad_fn=<MeanBackward1>))

In [74]:
a = torch.randn(2,2)
a = (a*3)/(a-1)
a

tensor([[-2.6610,  0.7082],
        [ 1.3297, -0.0783]])

In [75]:
a.requires_grad

False

In [76]:
a.requires_grad_(True)

tensor([[-2.6610,  0.7082],
        [ 1.3297, -0.0783]], requires_grad=True)

In [80]:
b = (a*a).sum()
b.grad_fn

In [81]:
b

tensor(9.3567, grad_fn=<SumBackward0>)

In [82]:
out.backward()

In [83]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [87]:
x = torch.randn(3, requires_grad=True)
x

tensor([ 1.4525, -0.0061,  0.8273], requires_grad=True)

In [99]:
a = torch.randn(2,2)
a.view(-1, 4)

tensor([[ 0.5100,  0.8190, -0.4715, -0.3571]])

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
#         self.conv1 = nn.Conv2d(1, 6, 3)
#         self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, x):
        # Max pooling over a (2, 2) window
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
#         # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=6, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=8, bias=True)
)


In [38]:
import numpy as np
input1 = torch.randn(2,6)
input1

tensor([[-1.3013,  1.4715, -0.6031, -0.5811, -2.8922, -0.9109],
        [ 0.0289, -0.7677,  0.9587,  0.6810,  0.6990,  0.5806]])

In [43]:
output = net(input1)
output

tensor([[-0.3021,  0.0750,  0.0989,  0.0822,  0.1944, -0.0406, -0.2482,  0.1078],
        [-0.1298,  0.0579,  0.1563, -0.0523,  0.1023,  0.0600,  0.0234,  0.1109]],
       grad_fn=<AddmmBackward>)

In [33]:
from torch.autograd import Variable
Variable(torch.from_numpy(input1), requires_grad=True)

tensor([[-0.8148],
        [-0.4074],
        [-0.1470],
        [ 0.4819],
        [-2.1844],
        [-0.2829]], dtype=torch.float64, requires_grad=True)

In [27]:
output.view(1,-1)

tensor([[-0.1243,  0.1170, -0.1777, -0.0916,  0.0502, -0.0126, -0.1282,  0.1006,
          0.0773, -0.3307]], grad_fn=<ViewBackward>)

In [40]:
output = net(input1)
target = torch.randn(6)  # a dummy target, for example
#target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)



RuntimeError: The size of tensor a (8) must match the size of tensor b (6) at non-singleton dimension 1

In [34]:
loss.data

tensor(0.4549)

In [22]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [6]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward(retain_graph=True)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0143,  0.0429,  0.0069, -0.0132,  0.0115,  0.0051])


In [9]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers

In [0]:
"""
This part of code is the Deep Q Network (DQN) brain.
view the tensorboard picture about this DQN structure on: https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/4-3-DQN3/#modification
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
Using:
Tensorflow: r1.2
"""

import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.set_random_seed(1)


# Deep Q Network off-policy
class DeepQNetwork:
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.01,
            reward_decay=0.9,
            e_greedy=0.9,
            replace_target_iter=300,
            memory_size=500,
            batch_size=32,
            e_greedy_increment=None,
            output_graph=False,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max

        # total learning step
        self.learn_step_counter = 0

        # initialize zero memory [s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features * 2 + 2))

        # consist of [target_net, evaluate_net]
        self._build_net()

        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='eval_net')

        with tf.variable_scope('hard_replacement'):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()

        if output_graph:
            # $ tensorboard --logdir=logs
            tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.cost_his = []

    def _build_net(self):
        # ------------------ all inputs ------------------------
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input State
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')  # input Next State
        self.r = tf.placeholder(tf.float32, [None, ], name='r')  # input Reward
        self.a = tf.placeholder(tf.int32, [None, ], name='a')  # input Action

        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)

        # ------------------ build evaluate_net ------------------
        with tf.variable_scope('eval_net'):
            e1 = tf.layers.dense(self.s, 20, tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='e1')
            self.q_eval = tf.layers.dense(e1, self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='q')

        # ------------------ build target_net ------------------
        with tf.variable_scope('target_net'):
            t1 = tf.layers.dense(self.s_, 20, tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='t1')
            self.q_next = tf.layers.dense(t1, self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='t2')

        with tf.variable_scope('q_target'):
            q_target = self.r + self.gamma * tf.reduce_max(self.q_next, axis=1, name='Qmax_s_')    # shape=(None, )
            self.q_target = tf.stop_gradient(q_target)
        with tf.variable_scope('q_eval'):
            a_indices = tf.stack([tf.range(tf.shape(self.a)[0], dtype=tf.int32), self.a], axis=1)
            self.q_eval_wrt_a = tf.gather_nd(params=self.q_eval, indices=a_indices)    # shape=(None, )
        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval_wrt_a, name='TD_error'))
        with tf.variable_scope('train'):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1

    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder
        observation = observation[np.newaxis, :]

        if np.random.uniform() < self.epsilon:
            # forward feed the observation and get q value for every actions
            actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
            action = np.argmax(actions_value)
        else:
            action = np.random.randint(0, self.n_actions)
        return action

    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.target_replace_op)
            print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        _, cost = self.sess.run(
            [self._train_op, self.loss],
            feed_dict={
                self.s: batch_memory[:, :self.n_features],
                self.a: batch_memory[:, self.n_features],
                self.r: batch_memory[:, self.n_features + 1],
                self.s_: batch_memory[:, -self.n_features:],
            })

        self.cost_his.append(cost)

        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1

    def plot_cost(self):
        import matplotlib.pyplot as plt
        plt.plot(np.arange(len(self.cost_his)), self.cost_his)
        plt.ylabel('Cost')
        plt.xlabel('training steps')
        plt.show()

# if __name__ == '__main__':
#     DQN = DeepQNetwork(3,4, output_graph=True)

In [0]:
import numpy as np
import pandas as pd

class Area():
    def __init__(self, n, a_id):
        self.a_id = a_id
        self.normal_bike = n
        self.broken_bike = 0

    def move(self):
        self.normal_bike -= 1
        self.broken_bike += 1
        
    def repair(self):
        self.normal_bike += 1
        self.broken_bike -= 1


def binaryInsert(target, events):
    for event in events:
        if event >= target[-1]:
            target.append(event)
        else:
            l, mid, r = 0, int(len(target) / 2), len(target) - 1
            while 1:
                if r - l == 1:
                    target.insert(r, event)
                    break
                else:
                    if event > target[mid]:
                        l = mid
                        mid = int((r + l) / 2)
                    else:
                        r = mid
                        mid = int((r + l) / 2)


class BikeNet():
    def __init__(self, N, R, A, Q, repair, P, time_limit):
        self.N = N
        self.R = R
        self.A = A
        self.Q = Q
        self.repair = repair
        self.P = P
        self.time_limit = time_limit
        self.reset()
        self.trans = {}

    def reset(self):

        # initiation of instances of Area and scheduler
        self.T = 0
        self.carrier_position = 0
        self.scheduler = []
        self.a = []  # list of instances of areas
        self.s = np.array([(self.N / self.A) if i % 2 == 0 else 0 for i in range(2 * self.A)])
        for i in range(A):
            self.a.append(Area(self.N / self.A, i))
            self.scheduler.append([np.random.exponential(1 / self.R.loc[i].cus_arr), 1, self.a[i]])
        self.scheduler.sort()

        return self.s.copy()

    def step(self, action):
        # time for carrier to take the action and repair one bicycle
        t = (np.abs(self.carrier_position % 3 - action % 3) + np.abs(self.carrier_position // 3 - action // 3)) / \
            self.R.loc[0].ride + self.repair
        t_cursor = self.T + t
        self.carrier_position = action
        reward = 0

        # update the atate of QN during the tansformation time
        while self.T < t_cursor:
            self.T = self.scheduler[0][0]
            if self.scheduler[0][1] == 1:
                # stat.loc[S], S = [scheduler[0][1], scheduler[0][2].a_id, T], S+1
                if self.scheduler[0][2].normal_bike == 0:
                    # this is a loss
                    reward -= 1
                    # loss.loc[L], L = [scheduler[0][2].a_id, self.T], L+1
                    event = [self.T + np.random.exponential(1 / self.R.loc[self.scheduler[0][2].a_id].cus_arr), 1,
                             self.scheduler[0][2]]
                    binaryInsert(self.scheduler, [event])
                else:
                    target = np.random.choice(np.arange(self.A + 1), 1, p=self.Q[self.scheduler[0][2].a_id])
                    if target == self.A:
                        # broken.loc[B], B = [self.scheduler[0][2].a_id, self.scheduler[0][2].normal_bike, self.scheduler[0][2].broken_bike, T], B+1
                        self.scheduler[0][2].move()
                        self.s[self.scheduler[0][2].a_id * 2], self.s[self.scheduler[0][2].a_id * 2 + 1] = \
                        self.scheduler[0][2].normal_bike, self.scheduler[0][2].broken_bike
                        continue
                    else:
                        self.scheduler[0][2].normal_bike -= 1
                        self.s[self.scheduler[0][2].a_id * 2] -= 1
                        event1 = [self.T + np.random.exponential(1 / self.R.loc[self.scheduler[0][2].a_id].ride), 2,
                                  self.a[target[0]]]
                        event2 = [self.T + np.random.exponential(1 / self.R.loc[self.scheduler[0][2].a_id].cus_arr), 1,
                                  self.scheduler[0][2]]
                        binaryInsert(self.scheduler, [event1, event2])
            else:
                # stat.loc[S], S = [scheduler[0][1], scheduler[0][2].a_id, T], S+1
                self.scheduler[0][2].normal_bike += 1
                self.s[self.scheduler[0][2].a_id * 2] += 1
            self.scheduler.pop(0)
        
        self.a[action].repair()
        s_ = self.s.copy()

        self.T = t_cursor
        if self.T < self.time_limit:
            return s_, reward, 0
        else:
            return s_, reward, 1

In [0]:
# from maze_env import Maze
# from RL_brain import DeepQNetwork
from tqdm import tqdm
from numba import jit


@jit
def simulate():
    #step = 0
    n_episodes = 500
    result = []
    for episode in tqdm(range(n_episodes)):
        sum_r = 0
        # initial observation
        observation = env.reset()
        action = 0
        #observation = np.array(int(N/A)) #devide all the normal bikes to all the areas evenly at the beginning

        while True:
            # fresh env
            #env.render()

            # RL choose action based on observation
            if not env.a[action].broken_bike:
            #action = RL.choose_action(observation)
                action = (action+1)%A

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            #RL.store_transition(observation, action, reward, observation_)

#             if (step > 200) and (step % 5 == 0):
#                 RL.learn()
            #RL.learn()

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            if done:
                break
            #step += 1
            sum_r += reward
            
        result.append([episode, sum_r])

    # end of game
    print('learning over')
    return result


if __name__ == "__main__":
    # maze game
    np.random.seed(1)
    N = 80 #total number of bikes in the QN
    A = 4 #A for areas, indicates the number of areas and the action space
    R = pd.DataFrame({'cus_arr': [1] * A, 'ride': [0.5] * A}, index=range(A))
    Q = [np.random.rand(A) for i in range(A)]
    Q = [q / sum(q)*0.99 for q in Q]
    Q = [np.append(q, 0.01) for q in Q]
    #Q = [[0,0.9,0.1], [0.9,0,0.1]]
    t_repair = 5
    P = 0
    time_limit = 1800

    env = BikeNet(N, R, A, Q, t_repair, P, time_limit)
    
#     RL = DeepQNetwork(A, 2*A,
#                       learning_rate=0.01,
#                       reward_decay=0.9,
#                       e_greedy=0.9,
#                       replace_target_iter=200,
#                       memory_size=2000,
#                       # output_graph=True
#                       )
    output = simulate()
    #RL.plot_cost()

In [0]:
output